In [1]:
import pandas as pd
import itertools
'''
Get Seed and repeat
'''
# Generate seed C1
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return [frozenset(i) for i in C1]


#Scan through datasets, remove the ones below min support
def scanD(D, Ck, minSupport):
    # Generate counts for the C
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    #print('Cn with count:',pd.DataFrame.from_dict(ssCnt,orient='index'))

    # Check weather meet minimum support or not, remove the ones below
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]#/numItems
        #print('support',support,'min_support',minSupport,'Bool',support >= minSupport)
        if support >= minSupport*numItems:
            retList.insert(0, key)
            supportData[key] = support
    #print('supportData:',pd.DataFrame.from_dict(supportData,orient='index'))
    #print('-'*30)
    return retList, supportData

def notRedundant(mergedlist,Lkm1,km1,l1,l2):
    sub_mergedlist=[set(i) for i in itertools.combinations(mergedlist,km1) if set(i) not in [l1,l2]]
    for i in sub_mergedlist:
        if i not in Lkm1:
            return False
    return True


#Generate Ck according to Lk
def aprioriGen(Lk, k):
    #print('AprioriGen Lk',Lk)
    #print('AprioriGen k',k)
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[: k-2]
            L2 = list(Lk[j])[: k-2]
            L1.sort()
            L2.sort()
            #If the all elements exept the last are same, merge two elements
            if L1 == L2:
                mergedlist=Lk[i] | Lk[j]
                #print('---------','|',mergedlist,'|',k-1,'\n')
                #print('*Generate*','|',mergedlist,'|',k-1,'\n')
                if notRedundant(mergedlist,Lk,k-1,Lk[i],Lk[j]):
                    retList.append(mergedlist)
                #else:
                    #print('----Remove-----','|',mergedlist,'|',k-1,'\n')
                    #print('*Remove*','|',mergedlist,'|',k-1,'\n')
    return retList


def apriori(dataSet, minSupport=0.5):
    #Generate C1 
    C1 = createC1(dataSet)
    D=list(map(set,dataSet))
    L1, supportData = scanD(D, C1, minSupport)

    #print('supportData',pd.DataFrame(supportData.values(),supportData.keys()))
    #print('L1',(L1))
    #print('------Seed------'*5,'\n')

    #repeatively generate L and C
    L = [L1]
    k = 2 #help to put values in L
    while (len(L[k-2]) > 0):
        #print('L_',k-2,L[k-2])
        Ck = aprioriGen(L[k-2], k) 
        #print(pd.DataFrame({'Ck':Ck}))
        Lk, supK = scanD(D, Ck, minSupport) 
        #print(pd.DataFrame(supK.values(),supK.keys()))
        supportData.update(supK)
        #print('*'*10)
        if len(Lk) == 0:
            break
        # all the Lk generated along the way are put into L.
        L.append(Lk)
        k += 1
    return L, supportData


'''
Mine Association Rule
'''
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        #print('freqSet:',freqSet)
        #print('conseq:',conseq)
        #print('freqSet-conseq',freqSet-conseq)
        #print('conf:',conf)
        if conf >= minConf:
            #print(freqSet-conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet,freqSet-conseq, conseq,supportData[freqSet-conseq],supportData[freqSet],supportData[conseq]))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    #print('*'*10*m,'H[0]',H[0])
    #print('H:',H)
    if (len(freqSet) >(m+1)):
        H = calcConf(freqSet, H, supportData, brl, minConf)
        Hmp1 = aprioriGen(H, m+1)
        #print('*'*20*m,Hmp1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        #print('Hmp1=', Hmp1)
        #print('len(Hmp1)=', len(Hmp1), 'len(freqSet)=', len(freqSet))
        if (len(Hmp1) >= 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

# Generate Association rules
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:#wont calculate with frequent item=1
            H1 = [frozenset([item]) if isinstance(item,list) else frozenset(item) for item in freqSet]#################
            #print('H1',H1)
            if (i > 1):#if only has two items, no need to split the relation
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
            #print('-'*30)
    return bigRuleList

def printrules(rules,total):
    try:
        C=['Frequent Itemset(AUB)','From(A)','To(B)','Confident Number(num of A)','Support Number(num of AUB)','Support Number(num of B)']  
        Ruletable=pd.DataFrame(rules)
        Ruletable.columns=C
        Ruletable['Confidence']=Ruletable['Support Number(num of AUB)']/Ruletable['Confident Number(num of A)']
        Ruletable['Support']=Ruletable['Support Number(num of AUB)']/total
        Ruletable['Lift']=(Ruletable['Support Number(num of AUB)']/total)/(Ruletable['Support Number(num of B)']/total)/(Ruletable['Confident Number(num of A)']/total)
        #print(Ruletable.shape)
        Ruletable.drop_duplicates(inplace=True)
        Ruletable.reset_index(drop=True,inplace=True)
        #print(Ruletable.shape)
    except:
        raise
        #print('May not be able to generate rule')
        return
    return Ruletable

def printsupport(S):
    S_table=pd.DataFrame.from_dict(S,orient='index').reset_index()
    S_table.columns=['itemset','support']
    S_table
    return S_table

if __name__ == "__main__":
    dataSet = [['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]
    #dataSet=simpDat
    LI, S = apriori(dataSet, minSupport=1.9/9)
    rules = generateRules(LI, S, minConf=0)
    #printrules(rules,len(dataSet))


KeyError: frozenset({'3', 'I'})

In [1]:
simpDat=[]
with open('Example_data.txt', 'r') as f:
    for line in f.readlines():
        L=line.split(',')
        L[-1]=L[-1].replace('\n','')
        simpDat.append(L)

In [2]:
import pandas as pd
import itertools
'''
Get Seed and repeat
'''
# Generate seed C1
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return [frozenset(i) for i in C1]


#Scan through datasets, remove the ones below min support
def scanD(D, Ck, minSupport):
    # Generate counts for the C
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    #print('Cn with count:',pd.DataFrame.from_dict(ssCnt,orient='index'))

    # Check weather meet minimum support or not, remove the ones below
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]#/numItems
        #print('support',support,'min_support',minSupport,'Bool',support >= minSupport)
        if support >= minSupport*numItems:
            retList.insert(0, key)
            supportData[key] = support
    #print('supportData:',pd.DataFrame.from_dict(supportData,orient='index'))
    #print('-'*30)
    return retList, supportData

def notRedundant(mergedlist,Lkm1,km1,l1,l2):
    sub_mergedlist=[set(i) for i in itertools.combinations(mergedlist,km1) if set(i) not in [l1,l2]]
    for i in sub_mergedlist:
        if i not in Lkm1:
            return False
    return True


#Generate Ck according to Lk
def aprioriGen(Lk, k):
    #print('AprioriGen Lk',Lk)
    #print('AprioriGen k',k)
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[: k-2]
            L2 = list(Lk[j])[: k-2]
            L1.sort()
            L2.sort()
            #If the all elements exept the last are same, merge two elements
            if L1 == L2:
                mergedlist=Lk[i] | Lk[j]
                #print('---------','|',mergedlist,'|',k-1,'\n')
                #print('*Generate*','|',mergedlist,'|',k-1,'\n')
                if notRedundant(mergedlist,Lk,k-1,Lk[i],Lk[j]):
                    retList.append(mergedlist)
                #else:
                    #print('----Remove-----','|',mergedlist,'|',k-1,'\n')
                    #print('*Remove*','|',mergedlist,'|',k-1,'\n')
    return retList


def apriori(dataSet, minSupport=0.5):
    #Generate C1 
    C1 = createC1(dataSet)
    D=list(map(set,dataSet))
    L1, supportData = scanD(D, C1, minSupport)

    #print('supportData',pd.DataFrame(supportData.values(),supportData.keys()))
    #print('L1',(L1))
    #print('------Seed------'*5,'\n')

    #repeatively generate L and C
    L = [L1]
    k = 2 #help to put values in L
    while (len(L[k-2]) > 0):
        #print('L_',k-2,L[k-2])
        Ck = aprioriGen(L[k-2], k) 
        #print(pd.DataFrame({'Ck':Ck}))
        Lk, supK = scanD(D, Ck, minSupport) 
        #print(pd.DataFrame(supK.values(),supK.keys()))
        supportData.update(supK)
        #print('*'*10)
        if len(Lk) == 0:
            break
        # all the Lk generated along the way are put into L.
        L.append(Lk)
        k += 1
    return L, supportData


'''
Mine Association Rule
'''
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        #print('freqSet:',freqSet)
        #print('conseq:',conseq)
        #print('freqSet-conseq',freqSet-conseq)
        #print('conf:',conf)
        if conf >= minConf:
            #print(freqSet-conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet,freqSet-conseq, conseq,supportData[freqSet-conseq],supportData[freqSet],supportData[conseq]))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    #print('*'*10*m,'H[0]',H[0])
    #print('H:',H)
    if (len(freqSet) >(m+1)):
        H = calcConf(freqSet, H, supportData, brl, minConf)
        Hmp1 = aprioriGen(H, m+1)
        #print('*'*20*m,Hmp1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        #print('Hmp1=', Hmp1)
        #print('len(Hmp1)=', len(Hmp1), 'len(freqSet)=', len(freqSet))
        if (len(Hmp1) >= 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

# Generate Association rules
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:#wont calculate with frequent item=1
            H1 = [frozenset([item]) for item in freqSet]
            #print('H1',H1)
            if (i > 1):#if only has two items, no need to split the relation
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
            #print('-'*30)
    return bigRuleList

def printrules(rules,total):
    try:
        C=['Frequent Itemset(AUB)','From(A)','To(B)','Confident Number(num of A)','Support Number(num of AUB)','Support Number(num of B)']  
        Ruletable=pd.DataFrame(rules)
        Ruletable.columns=C
        Ruletable['Confidence']=Ruletable['Support Number(num of AUB)']/Ruletable['Confident Number(num of A)']
        Ruletable['Support']=Ruletable['Support Number(num of AUB)']/total
        Ruletable['Lift']=(Ruletable['Support Number(num of AUB)']/total)/(Ruletable['Support Number(num of B)']/total)/(Ruletable['Confident Number(num of A)']/total)
        #print(Ruletable.shape)
        Ruletable.drop_duplicates(inplace=True)
        Ruletable.reset_index(drop=True,inplace=True)
        #print(Ruletable.shape)
    except:
        raise
        #print('May not be able to generate rule')
        return
    return Ruletable

def printsupport(S):
    S_table=pd.DataFrame.from_dict(S,orient='index').reset_index()
    S_table.columns=['itemset','support']
    S_table
    return S_table

if __name__ == "__main__":
    #dataSet = [['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]
    dataSet=simpDat
    LI, S = apriori(dataSet, minSupport=0.5)
    print('Frequent List',LI)
    print(printsupport(S))
    rules = generateRules(LI, S, minConf=0)
    print(printrules(rules,len(dataSet)))


Frequent List [[frozenset({'High'}), frozenset({'capital_loss_1'}), frozenset({'White'}), frozenset({'capital_gain_1'}), frozenset({'United-States'}), frozenset({'Private'}), frozenset({'Male'}), frozenset({'<=50K.'})], [frozenset({'High', 'capital_loss_1'}), frozenset({'<=50K.', 'White'}), frozenset({'White', 'Male'}), frozenset({'Private', 'White'}), frozenset({'United-States', 'White'}), frozenset({'capital_gain_1', 'White'}), frozenset({'<=50K.', 'capital_loss_1'}), frozenset({'Male', 'capital_loss_1'}), frozenset({'Private', 'capital_loss_1'}), frozenset({'United-States', 'capital_loss_1'}), frozenset({'capital_gain_1', 'capital_loss_1'}), frozenset({'White', 'capital_loss_1'}), frozenset({'<=50K.', 'Private'}), frozenset({'United-States', '<=50K.'}), frozenset({'United-States', 'Male'}), frozenset({'United-States', 'Private'}), frozenset({'capital_gain_1', '<=50K.'}), frozenset({'capital_gain_1', 'Male'}), frozenset({'capital_gain_1', 'Private'}), frozenset({'capital_gain_1', 'Un

In [25]:
'''#step by step:
dataSet = [['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]'''
simpDat=[]
with open('Example_data.txt', 'r') as f:
    for line in f.readlines():
        L=line.split(',')
        L[-1]=L[-1].replace('\n','')
        simpDat.append(L)


In [4]:
printrules(rules,len(dataSet))


,Frequent Itemset(AUB),From(A),To(B),Confident Number(num of A),Support Number(num of AUB),Support Number(num of B),Confidence,Support,Lift
0,"(I1, I3)",(I3),(I1),6,4,6,0.666667,0.444444,1.000000
1,"(I1, I3)",(I1),(I3),6,4,6,0.666667,0.444444,1.000000
2,"(I3, I2)",(I2),(I3),7,4,6,0.571429,0.444444,0.857143
3,"(I3, I2)",(I3),(I2),6,4,7,0.666667,0.444444,0.857143
4,"(I4, I2)",(I2),(I4),7,2,2,0.285714,0.222222,1.285714
5,"(I4, I2)",(I4),(I2),2,2,7,1.000000,0.222222,1.285714
6,"(I1, I2)",(I2),(I1),7,4,6,0.571429,0.444444,0.857143
7,"(I1, I2)",(I1),(I2),6,4,7,0.666667,0.444444,0.857143
8,"(I1, I5)",(I5),(I1),2,2,6,1.000000,0.222222,1.500000
9,"(I1, I5)",(I1),(I5),6,2,2,0.333333,0.222222,1.500000


In [5]:
printsupport(S)

,itemset,support
0,(I1),6
1,(I2),7
2,(I5),2
3,(I4),2
4,(I3),6
5,"(I2, I5)",2
6,"(I1, I5)",2
7,"(I1, I2)",4
8,"(I4, I2)",2
9,"(I3, I2)",4


In [10]:
T=printrules(rules,len(dataSet))
T
#T.sort_value()
#T.sort_values('Frequent Itemset(AUB)',ascending=False).reset_index(drop=True)

,Frequent Itemset(AUB),From(A),To(B),Confident Number(num of A),Support Number(num of AUB),Support Number(num of B),Confidence,Support,Lift
0,"(capital_loss_1, High)",(High),(capital_loss_1),8843,8323,15517,0.941196,0.511209,0.987537
1,"(capital_loss_1, High)",(capital_loss_1),(High),15517,8323,8843,0.536379,0.511209,0.987537
2,"(<=50K., White)",(White),(<=50K.),13946,10456,12435,0.749749,0.642221,0.981638
3,"(<=50K., White)",(<=50K.),(White),12435,10456,13946,0.840852,0.642221,0.981638
4,"(White, Male)",(Male),(White),10860,9561,13946,0.880387,0.587249,1.027791
...,...,...,...,...,...,...,...,...,...
449,"(<=50K., White, United-States, capital_gain_1,...",(capital_loss_1),"(<=50K., White, United-States, capital_gain_1)",15517,8865,9176,0.571309,0.544500,1.013675
450,"(<=50K., White, United-States, capital_gain_1,...",(capital_gain_1),"(<=50K., White, United-States, capital_loss_1)",14958,8865,9273,0.592659,0.544500,1.040557
451,"(<=50K., White, United-States, capital_gain_1,...",(United-States),"(<=50K., White, capital_gain_1, capital_loss_1)",14662,8865,9689,0.604624,0.544500,1.015986
452,"(<=50K., White, United-States, capital_gain_1,...",(White),"(<=50K., United-States, capital_gain_1, capita...",13946,8865,10313,0.635666,0.544500,1.003518


In [46]:
setcounts={k:SelectT.loc[i,'Confident number(num of A)'] for i,k in enumerate(SelectT['From(A)'])}
Bcounts=[setcounts[i] for i in SelectT['To(B)']]
print(setcounts,Bcounts)

{frozenset({'I3'}): 6, frozenset({'I1'}): 6, frozenset({'I2'}): 7, frozenset({'I4'}): 2, frozenset({'I5'}): 2, frozenset({'I1', 'I3'}): 4, frozenset({'I2', 'I3'}): 4, frozenset({'I2', 'I1'}): 4, frozenset({'I1', 'I5'}): 2, frozenset({'I2', 'I5'}): 2} [6, 6, 7, 6, 7, 2, 7, 6, 6, 2, 7, 2, 7, 6, 6, 4, 4, 4, 7, 6, 2, 4, 2, 2]


In [18]:
printsupport(S)

,itemset,support
0,(I1),6
1,(I2),7
2,(I5),2
3,(I4),2
4,(I3),6
5,"(I5, I2)",2
6,"(I5, I1)",2
7,"(I1, I2)",4
8,"(I4, I2)",2
9,"(I3, I2)",4


In [56]:
C=['Frequent itemset','From','To','Confident number','Support Number']
Ruletable=pd.DataFrame(rules)
Ruletable.columns=C
Ruletable['Confidence']=Ruletable['Support Number']/Ruletable['Confident number']
Ruletable

,Frequent itemset,From,To,Confident number,Support Number,Confidence
0,"(I3, I1)",(I1),(I3),0.666667,0.444444,0.666667
1,"(I3, I1)",(I3),(I1),0.666667,0.444444,0.666667
2,"(I3, I2)",(I2),(I3),0.777778,0.444444,0.571429
3,"(I3, I2)",(I3),(I2),0.666667,0.444444,0.666667
4,"(I2, I4)",(I4),(I2),0.222222,0.222222,1.000000
5,"(I2, I4)",(I2),(I4),0.777778,0.222222,0.285714
6,"(I2, I1)",(I1),(I2),0.666667,0.444444,0.666667
7,"(I2, I1)",(I2),(I1),0.777778,0.444444,0.571429
8,"(I5, I1)",(I1),(I5),0.666667,0.222222,0.333333
9,"(I5, I1)",(I5),(I1),0.222222,0.222222,1.000000


In [25]:
#print(LI)

c=0
for i in LI:
    for ii in i:
        c+=1
        print(ii)
print(c)

frozenset({'I3'})
frozenset({'I4'})
frozenset({'I5'})
frozenset({'I2'})
frozenset({'I1'})
frozenset({'I3', 'I1'})
frozenset({'I3', 'I2'})
frozenset({'I2', 'I4'})
frozenset({'I2', 'I1'})
frozenset({'I5', 'I1'})
frozenset({'I5', 'I2'})
frozenset({'I3', 'I2', 'I1'})
frozenset({'I5', 'I2', 'I1'})
13


In [10]:
with_S=pd.DataFrame(S.values(),S.keys()).reset_index()
with_S[0]=with_S[0]*len(dataSet)
with_S


,index,0
0,(I1),6.0
1,(I2),7.0
2,(I5),2.0
3,(I4),2.0
4,(I3),6.0
5,"(I5, I2)",2.0
6,"(I5, I1)",2.0
7,"(I2, I1)",4.0
8,"(I2, I4)",2.0
9,"(I3, I2)",4.0
